In [1]:
import sys
sys.path.append('/home/witoon/thesis/code/Soft-Actor-Critic')

In [2]:
import os

import numpy as np
import torchvision.transforms as T
import matplotlib.pyplot as plt

from itertools import cycle
from PIL import Image, ImageDraw, ImageFont

from common.network import ConvBetaVAE

/home/witoon/.venv/ml38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
image_size = (256, 512)
H, W = image_size

In [4]:
weight_base_path = '../vae_weights'
models_dict = {
#     'b1': (1, 'Carla-v0_town7_outskirts_b1//bvae_town7_epoch(50)-loss(+2.503E+05).pkl'),
    'b2': (2, 'Carla-v0_town7_b3_new_tanh_mse_flip/bvae_town7_epoch(100)-loss(+8.586E+04).pkl'),
    'b3': (3, 'Carla-v0_town7_b3_new_tanh_mse/bvae_town7_epoch(95)-loss(+8.734E+04).pkl'),
#     'b4': (4, 'Carla-v0_town7_outskirts_b4//bvae_town7_epoch(100)-loss(+2.455E+05).pkl')
}

In [5]:
base_path = '../vae/eval-images'
image_paths = list(map(lambda name: os.path.join(base_path, name), os.listdir(base_path)))

In [6]:
def denormalize_image(np_img):
    mean = np.array([0.4652, 0.4417, 0.3799])
    std = np.array([0.0946, 0.1767, 0.1865])
    
    new_img = ((np_img * std) + mean) * 255
#     new_img = np_img * 255
    return new_img.astype('uint8')

In [7]:
to_tensor = T.Compose([
    T.ToTensor(),
#     T.Normalize(mean=[0.4640, 0.4763, 0.3560], std=[0.0941, 0.1722, 0.1883])
])

def out_tensor_to_np_image(out_tensor):
    ''' Convert output tensor to numpy image '''
    out_img_np = out_tensor[0].squeeze(dim=0).detach().numpy()
    out_img_np = denormalize_image(out_img_np.transpose((1, 2, 0)))
    return out_img_np

In [8]:
reconstruct_images = []
for key, (beta, weight_name) in models_dict.items():
    latent_size = 512
    model = ConvBetaVAE(image_size, latent_size=latent_size, beta=beta)
    model.load_model(os.path.join(weight_base_path, weight_name))
    
    images = []
    for image_path in image_paths:
        pil_img = Image.open(image_path)
        np_img = np.array(pil_img)
        in_tensor = to_tensor(pil_img)
        
        out_tensor = model(in_tensor.unsqueeze(dim=0))
        out_np_img = out_tensor_to_np_image(out_tensor)
        
        combined_np_img = np.hstack([np_img, out_np_img])
        images.append(combined_np_img)
        
        pil_img.close()
        
    reconstruct_images.append(images)

In [9]:
for num, images in enumerate(zip(*reconstruct_images), 1):
    combined_np_img = np.vstack(images)
    combined_pil_img = Image.fromarray(combined_np_img)

    draw = ImageDraw.Draw(combined_pil_img)
    
    pos_y = 0
    margin = 10
    
    for beta in range(2, 4):
        text = f'Beta {beta}'
        # textwidth, textheight = draw.textsize(text)

        pos_y = pos_y + margin
        draw.text((10, pos_y), text)
        
        pos_y += H

    combined_pil_img.save(f'../vae/visualize-result/tanh-mse/visualize-result-{num}.jpeg')